Make sure Python and Jupyter are set up locally first (and if necessary relaunch VSCode). There is a `shell.nix` that does this for you if you have Nix installed, or you can use a virtual env created from the global Python.

```bash
$ python3 -m venv .venv
$ . .venv/bin/activate
$ pip install jupyter ipykernel notebook
$ pip install -r requirements.txt
```

Some LLM code copied from [Qwak.com](https://www.qwak.com/post/utilizing-llms-with-embedding-stores#building-a-closed-qa-bot-with-falcon-7b-and-chromadb)...

In [1]:
from datasets import load_dataset

# Load only the training split of the dataset
train_dataset = load_dataset("databricks/databricks-dolly-15k", split='train')

# Filter the dataset to only include entries with the 'closed_qa' category
closed_qa_dataset = train_dataset.filter(lambda example: example['category'] == 'closed_qa')

print(closed_qa_dataset[0])

{'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}


Make sure ollama is running in the background and the model is available:

```bash
$ ollama serve
$ ollama pull mistral
$ ollama pull albertogg/multi-qa-minilm-l6-cos-v1
```

(You can tell it to pull a model from Python too.)

In [1]:
import chromadb

# Initialize the embedding model
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="knowledge-base")

In [4]:
import ollama

# Method to populate the vector store with embeddings from a dataset
def populate_vectors(dataset):
    for i, item in enumerate(dataset):
        combined_text = f"{item['instruction']} {item['context']}"
        embeddings = ollama.embeddings(model='albertogg/multi-qa-minilm-l6-cos-v1', prompt=combined_text)['embedding']
        collection.add(embeddings=[embeddings], documents=[item['context']], ids=[f"id_{i}"])

# Method to search the ChromaDB collection for relevant context based on a query
def search_context(query, n_results=1):
    query_embeddings = ollama.embeddings(model='albertogg/multi-qa-minilm-l6-cos-v1', prompt=query)['embedding']
    return collection.query(query_embeddings=query_embeddings, n_results=n_results)

In [4]:
populate_vectors(closed_qa_dataset)

In [7]:
instruction = "What is the name of the major school of praxiology not developed by Ludwig von Mises"
context = "In philosophy, praxeology or praxiology (/\u02ccpr\u00e6ksi\u02c8\u0252l\u0259d\u0292i/; from Ancient Greek \u03c0\u03c1\u1fb6\u03be\u03b9\u03c2 (praxis) 'deed, action', and -\u03bb\u03bf\u03b3\u03af\u03b1 (-logia) 'study of') is the theory of human action, based on the notion that humans engage in purposeful behavior, contrary to reflexive behavior and other unintentional behavior.\n\nFrench social philosopher Alfred Espinas gave the term its modern meaning, and praxeology was developed independently by two principal groups: the Austrian school, led by Ludwig von Mises, and the Polish school, led by Tadeusz Kotarbi\u0144ski."
ollama.embed(model='albertogg/multi-qa-minilm-l6-cos-v1', input=f"Content: {instruction} {context}")

{'model': 'albertogg/multi-qa-minilm-l6-cos-v1',
 'embeddings': [[-0.047326926,
   0.04261045,
   -0.04876651,
   -0.06244526,
   0.020464458,
   -0.048344012,
   -0.04408791,
   -0.002949139,
   0.0025748874,
   0.05482461,
   0.11849957,
   0.025493044,
   0.014819109,
   0.06831706,
   0.009105153,
   -0.046532623,
   -0.16621193,
   0.05230968,
   0.010075348,
   0.023884395,
   0.015539375,
   0.0028502217,
   0.060684662,
   -0.062685415,
   -0.046413578,
   -0.048255168,
   0.00030686293,
   -0.12090608,
   0.018961545,
   0.074324206,
   0.11006716,
   6.0693263e-05,
   0.061112206,
   -0.048079386,
   -0.025828438,
   0.04921832,
   -0.024565157,
   -0.03051368,
   -0.0028907557,
   0.097533785,
   -0.06472982,
   0.061170023,
   -0.0089735575,
   0.020445723,
   -0.005381917,
   0.0071884934,
   -0.0033629434,
   0.057539146,
   -0.04797672,
   0.013695918,
   -0.061673347,
   -0.056127004,
   -0.026096934,
   -0.03828968,
   -0.055517305,
   0.03823934,
   -0.00991792,
   -0

In [5]:
closed_qa_dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 1773
})

In [2]:
collection.get_model()

Collection(id=UUID('ff8dce2d-b4f2-4eae-bcb7-579cc1391a6c'), name='knowledge-base', configuration_json={'hnsw_configuration': {'space': 'l2', 'ef_construction': 100, 'ef_search': 10, 'num_threads': 8, 'M': 16, 'resize_factor': 1.2, 'batch_size': 100, 'sync_threshold': 1000, '_type': 'HNSWConfigurationInternal'}, '_type': 'CollectionConfigurationInternal'}, metadata=None, dimension=None, tenant='default_tenant', database='default_database', version=0)

In [8]:
collection.query(query_embeddings=[ollama.embeddings(model='albertogg/multi-qa-minilm-l6-cos-v1', prompt='When was Tomoaki Komorida born?')['embedding']], n_results=1)

{'ids': [[]],
 'distances': [[]],
 'metadatas': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [10]:
user_question = "When was Tomoaki Komorida born?"
context_response = search_context(user_question)
context = "".join(context_response['documents'][0])
context

'Komorida was born in Kumamoto Prefecture on July 10, 1981. After graduating from high school, he joined the J1 League club Avispa Fukuoka in 2000. Although he debuted as a midfielder in 2001, he did not play much and the club was relegated to the J2 League at the end of the 2001 season. In 2002, he moved to the J2 club Oita Trinita. He became a regular player as a defensive midfielder and the club won the championship in 2002 and was promoted in 2003. He played many matches until 2005. In September 2005, he moved to the J2 club Montedio Yamagata. In 2006, he moved to the J2 club Vissel Kobe. Although he became a regular player as a defensive midfielder, his gradually was played less during the summer. In 2007, he moved to the Japan Football League club Rosso Kumamoto (later Roasso Kumamoto) based in his local region. He played as a regular player and the club was promoted to J2 in 2008. Although he did not play as much, he still played in many matches. In 2010, he moved to Indonesia a

In [19]:
prompt = f"""
{user_question}

Context information is below.
---------------------
{context}
---------------------
Given the context and provided history information and not prior knowledge,
reply to the user comment. If the answer is not in the context, inform
the user that you can't answer the question.
"""

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [11]:
response = ollama.chat(model="mistral", messages=[{'role':'user', 'content': f"{context}\n\n{user_question}"}])
response['message']['content'].strip()

'Komorida was born on July 10, 1981.'

In [13]:
for i in range(10):
	response = ollama.chat(model="mistral", messages=[{'role':'user', 'content': f"{context}\n\n{user_question}"}])
	print(response['message']['content'].strip())

Tomoaki Komorida was born on July 10, 1981.
Tomoaki Komorida was born on July 10, 1981.
Tomoaki Komorida was born on July 10, 1981.
Tomoaki Komorida was born on July 10, 1981.
Tomoaki Komorida was born on July 10, 1981.
Tomoaki Komorida was born on July 10, 1981.
Tomoaki Komorida was born on July 10, 1981.
Komorida was born on July 10, 1981.
Tomoaki Komorida was born on July 10, 1981.
Tomoaki Komorida was born on July 10, 1981.


In [37]:
ollama.generate(model='mistral', prompt=f"{context}\n\n{user_question}")['response'].strip()

'Tomoaki Komorida was born on July 10, 1981.'

In [13]:
import ollama
ollama.embed(model='mistral', input="Hello World")

{'model': 'mistral',
 'embeddings': [[0.017545225,
   -0.008891564,
   0.019453695,
   0.0016723485,
   0.0066151437,
   -0.011255239,
   0.0073642163,
   0.015485475,
   0.0030973046,
   -0.002989305,
   -0.00017289614,
   -0.0051300055,
   -0.009368509,
   -0.0050006732,
   0.019989194,
   -0.00912649,
   0.031789336,
   -0.0003259816,
   0.0028014206,
   -0.030878957,
   -0.010182084,
   -0.005244998,
   0.0028699695,
   0.018529905,
   0.0035672926,
   -0.005899601,
   0.008602345,
   -0.016584355,
   0.004158217,
   -0.03023776,
   -0.003201344,
   -0.0077145896,
   -0.0022898177,
   -0.02077727,
   0.016491277,
   -0.009456201,
   0.007900923,
   -0.0021981897,
   -0.02405443,
   3.098758e-05,
   0.020625295,
   -0.011458679,
   -0.0051068566,
   -0.0085069295,
   0.017706133,
   0.009591857,
   0.00090422056,
   -0.010826694,
   0.0070333797,
   0.0053707818,
   0.0048018754,
   0.0034794498,
   0.011210644,
   -0.0903977,
   0.0018821944,
   -0.0018045873,
   -0.030915277,
   0

In [13]:
%%bash
pip install pipreqs
jupyter nbconvert --to=python README.ipynb
pipreqs --ignore ".venv" .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 11.0 MB/s eta 0:00:00m eta 0:00:010:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=0d029a8ff4638c40fa356fa0811eb9062726c077827fea0c117d5162219a1c1b
  Stored in directory: /home/dsyer/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt
  Attempting uninstall: ipython
    Found existing installation: ipython 8.26.0
    Uninstalling ipython-8.26.0:
      Successfully uninstalled ipython-8.26.0

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
pd.read_json("data/lines.jsonl", lines=True)

,instruction,context,response
0,How to make a cup of tea?,,Boil water. Add tea bag. Pour water into cup. ...
1,What is the capital of France?,,Paris
2,What is the capital of Germany?,,Berlin
3,What is the capital of Italy?,,Rome
4,What is the capital of Spain?,,Madrid
5,What is the capital of Portugal?,,Lisbon
6,What is the capital of Greece?,,Athens
7,What is the capital of Turkey?,,Ankara
8,What is the capital of Egypt?,,Cairo
9,What is the capital of South Africa?,,Pretoria


In [10]:
data = load_dataset("./data", split="train")

In [12]:
data.__class__


datasets.arrow_dataset.Dataset